**DATA PRE-PROCESSING**

In [ ]:
!pip install nltk
!pip install bert-extractive-summarizer
!pip install transformers
!pip install SentencePiece
!pip install rouge

!pip install bertopic
!pip install top2vec
!pip install top2vec[sentence_encoders]
!pip install top2vec[indexing]

     |████████████████████████████████| 3.1 MB 8.3 MB/s 
     |████████████████████████████████| 596 kB 55.6 MB/s 
     |████████████████████████████████| 59 kB 7.2 MB/s 
     |████████████████████████████████| 3.3 MB 38.2 MB/s 
     |████████████████████████████████| 895 kB 74.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 7.6 MB/s 
     |████████████████████████████████| 57 kB 3.4 MB/s 
     |████████████████████████████████| 15.7 MB 17.3 MB/s 
     |████████████████████████████████| 78 kB 7.0 MB/s 
     |████████████████████████████████| 636 kB 64.2 MB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
     |████████████████████████████████| 13.2 MB 18.5 MB/s 
     |████████████████████████████████| 6.4 MB 16.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing whe

     |████████████████████████████████| 4.9 MB 8.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hnswlib: filename=hnswlib-0.5.2-cp37-cp37m-linux_x86_64.whl size=1326184 sha256=e6dfa134444de0cab346733adfd7f6894b53ec8b688015dd707b8545995f28a0
  Stored in directory: /root/.cache/pip/wheels/b4/11/b3/337c4a361b31217d62c3b420ad66fe20d381f1ebb29b046095
Successfully built hnswlib


In [ ]:
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
#from keras.preprocessing.text import Tokenizer 
#from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data=pd.read_pickle("/content/drive/MyDrive/pickle files/data_crawled_final.pkl") #Enter your filepath

In [ ]:
data = data[:100]

In [ ]:
data

,celex,summary,text
0,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nRegulation (EU) 2020/740 of the European Parliament and of the Council of 25 May 2020 on the labelling of tyres with respect to fuel efficiency and other paramet...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n5.6.2020 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 177/1\n\n\n\n\n\n\nREGULATION (EU) 2020/740 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof 25 ...
1,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nDirective (EU) 2019/944 of the European Parliament and of the Council of 5 June 2019 on common rules for the internal market for electricity and amending Directi...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n14.6.2019 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 158/125\n\n\n\n\n\n\nDIRECTIVE (EU) 2019/944 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof 5...
2,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nRegulation (EU) 2019/941 of the European Parliament and of the Council of 5 June 2019 on risk-preparedness in the electricity sector and repealing Directive 2005...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n14.6.2019 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 158/1\n\n\n\n\n\n\nREGULATION (EU) 2019/941 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof 5 ...
3,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nRegulation (EU) 2019/943 of the European Parliament and of the Council of 5 June 2019 on the internal market for electricity (Text with EEA relevance.)\nRegulati...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n14.6.2019 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 158/54\n\n\n\n\n\n\nREGULATION (EU) 2019/943 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof 5...
4,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nRegulation (EU) 2019/942 of the European Parliament and of the Council of 5 June 2019 establishing a European Union Agency for the Cooperation of Energy Regulato...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n14.6.2019 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 158/22\n\n\n\n\n\n\nREGULATION (EU) 2019/942 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof 5...
...,...,...,...
95,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nCommission Regulation (EU) 2016/631 of 14 April 2016 establishing a network code on requirements for grid connection of generators (Text with EEA relevance)\nCom...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n27.4.2016 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 112/1\n\n\n\n\n\nCOMMISSION REGULATION (EU) 2016/631\nof 14 April 2016\nestablishing a network c...
96,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nCommission Delegated Regulation (EU) 2015/2402 of 12 October 2015 reviewing harmonised efficiency reference values for separate production of electricity and hea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n19.12.2015 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 333/54\n\n\n\n\n\nCOMMISSION DELEGATED REGULATION (EU) 2015/2402\nof 12 October 2015\nreviewing...
97,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nCommission Opinion of 4 December 2015 relating to the plan for the disposal of radioactive waste arising from the decommissioning and the dismantling of the La H...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n5.12.2015 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nC 405/1\n\n\n\n\n\nCOMMISSION OPINION\nof 4 December 2015\nrelating to the plan for the disposal o...
98,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nCommission Implementing Decision (EU) 2015/1960 of 29 October 2015 on the establishment of the annual priority lists for 2016 for the development of network code...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n30.10.2015 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 284/187\n\n\n\n\n\nCOMMISSION IMPLEMENTING DECISION (EU) 2015/1960\nof 29 October 2015\non the ...


In [ ]:
data.drop_duplicates(subset=['text'],inplace=True)#dropping duplicates
data.dropna(axis=0,inplace=True)#dropping na

In [ ]:
data

,celex,summary,text
0,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nRegulation (EU) 2020/740 of the European Parliament and of the Council of 25 May 2020 on the labelling of tyres with respect to fuel efficiency and other paramet...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n5.6.2020 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 177/1\n\n\n\n\n\n\nREGULATION (EU) 2020/740 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof 25 ...
1,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nDirective (EU) 2019/944 of the European Parliament and of the Council of 5 June 2019 on common rules for the internal market for electricity and amending Directi...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n14.6.2019 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 158/125\n\n\n\n\n\n\nDIRECTIVE (EU) 2019/944 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof 5...
2,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nRegulation (EU) 2019/941 of the European Parliament and of the Council of 5 June 2019 on risk-preparedness in the electricity sector and repealing Directive 2005...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n14.6.2019 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 158/1\n\n\n\n\n\n\nREGULATION (EU) 2019/941 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof 5 ...
3,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nRegulation (EU) 2019/943 of the European Parliament and of the Council of 5 June 2019 on the internal market for electricity (Text with EEA relevance.)\nRegulati...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n14.6.2019 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 158/54\n\n\n\n\n\n\nREGULATION (EU) 2019/943 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof 5...
4,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nRegulation (EU) 2019/942 of the European Parliament and of the Council of 5 June 2019 establishing a European Union Agency for the Cooperation of Energy Regulato...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n14.6.2019 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 158/22\n\n\n\n\n\n\nREGULATION (EU) 2019/942 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof 5...
...,...,...,...
95,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nCommission Regulation (EU) 2016/631 of 14 April 2016 establishing a network code on requirements for grid connection of generators (Text with EEA relevance)\nCom...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n27.4.2016 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 112/1\n\n\n\n\n\nCOMMISSION REGULATION (EU) 2016/631\nof 14 April 2016\nestablishing a network c...
96,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nCommission Delegated Regulation (EU) 2015/2402 of 12 October 2015 reviewing harmonised efficiency reference values for separate production of electricity and hea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n19.12.2015 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 333/54\n\n\n\n\n\nCOMMISSION DELEGATED REGULATION (EU) 2015/2402\nof 12 October 2015\nreviewing...
97,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nCommission Opinion of 4 December 2015 relating to the plan for the disposal of radioactive waste arising from the decommissioning and the dismantling of the La H...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n5.12.2015 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nC 405/1\n\n\n\n\n\nCOMMISSION OPINION\nof 4 December 2015\nrelating to the plan for the disposal o...
98,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nCommission Implementing Decision (EU) 2015/1960 of 29 October 2015 on the establishment of the annual priority lists for 2016 for the development of network code...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n30.10.2015 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 284/187\n\n\n\n\n\nCOMMISSION IMPLEMENTING DECISION (EU) 2015/1960\nof 29 October 2015\non the ...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   celex    100 non-null    object
 1   summary  100 non-null    object
 2   text     100 non-null    object
dtypes: object(3)
memory usage: 3.1+ KB


In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = set(stopwords.words('english')) 

def text_cleaner(text):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    newString = re.sub('[m]{2,}', 'mm', newString)
    # if(num==0):
    #     tokens = [w for w in newString.split() if not w in stop_words]
    # else:
    tokens=newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:                                                 #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
cleaned_text = []
for t in data['text']:
    cleaned_text.append(text_cleaner(t))

In [ ]:
cleaned_text[:5]

['en official journal of the european union regulation of the european parliament and of the council of may on the labelling of tyres with respect to fuel efficiency and other parameters amending regulation and repealing regulation no the european parliament and the council of the european union having regard to the treaty on the functioning of the european union and in particular article and article thereof having regard to the proposal from the european commission after transmission of the draft legislative act to the national parliaments having regard to the opinion of the european economic and social committee after consulting the committee of the regions acting in accordance with the ordinary legislative procedure whereas the union is committed to building an energy union with forward looking climate policy fuel efficiency is crucial element of the union climate and energy policy framework and is key to moderating energy demand the commission has reviewed regulation no of the euro

In [ ]:
data['summary'][5]

'\n\n\t\t\t\t\t\t\t\u200b\n\t\t\t\t\t\t\nRegulation (EU) 2018/1999 of the European Parliament and of the Council of 11 December 2018 on the Governance of the Energy Union and Climate Action, amending Regulations (EC) No 663/2009 and (EC) No 715/2009 of the European Parliament and of the Council, Directives 94/22/EC, 98/70/EC, 2009/31/EC, 2009/73/EC, 2010/31/EU, 2012/27/EU and 2013/30/EU of the European Parliament and of the Council, Council Directives 2009/119/EC and (EU) 2015/652 and repealing Regulation (EU) No 525/2013 of the European Parliament and of the Council (Text with EEA relevance.)\nRegulation (EU) 2018/1999 of the European Parliament and of the Council of 11 December 2018 on the Governance of the Energy Union and Climate Action, amending Regulations (EC) No 663/2009 and (EC) No 715/2009 of the European Parliament and of the Council, Directives 94/22/EC, 98/70/EC, 2009/31/EC, 2009/73/EC, 2010/31/EU, 2012/27/EU and 2013/30/EU of the European Parliament and of the Council, Co

In [ ]:
#call the function
cleaned_summary = []
for t in data['summary']:
    cleaned_summary.append(text_cleaner(t))

In [ ]:
cleaned_summary[:5]

['regulation of the european parliament and of the council of may on the labelling of tyres with respect to fuel efficiency and other parameters amending regulation and repealing regulation no regulation of the european parliament and of the council of may on the labelling of tyres with respect to fuel efficiency and other parameters amending regulation and repealing regulation no regulation of the european parliament and of the council of may on the labelling of tyres with respect to fuel efficiency and other parameters amending regulation and repealing regulation no pe init oj in force this act has been changed current consolidated version eli http data europa eu eli reg oj',
 'directive of the european parliament and of the council of june on common rules for the internal market for electricity and amending directive eu directive of the european parliament and of the council of june on common rules for the internal market for electricity and amending directive eu directive of the eu

In [ ]:
data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary

In [ ]:
data[:10]

,celex,summary,text,cleaned_text,cleaned_summary
0,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nRegulation (EU) 2020/740 of the European Parliament and of the Council of 25 May 2020 on the labelling of tyres with respect to fuel efficiency and other paramet...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n5.6.2020 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 177/1\n\n\n\n\n\n\nREGULATION (EU) 2020/740 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof 25 ...,en official journal of the european union regulation of the european parliament and of the council of may on the labelling of tyres with respect to fuel efficiency and other parameters amending re...,regulation of the european parliament and of the council of may on the labelling of tyres with respect to fuel efficiency and other parameters amending regulation and repealing regulation no regul...
1,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nDirective (EU) 2019/944 of the European Parliament and of the Council of 5 June 2019 on common rules for the internal market for electricity and amending Directi...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n14.6.2019 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 158/125\n\n\n\n\n\n\nDIRECTIVE (EU) 2019/944 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof 5...,en official journal of the european union directive of the european parliament and of the council of june on common rules for the internal market for electricity and amending directive eu the euro...,directive of the european parliament and of the council of june on common rules for the internal market for electricity and amending directive eu directive of the european parliament and of the co...
2,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nRegulation (EU) 2019/941 of the European Parliament and of the Council of 5 June 2019 on risk-preparedness in the electricity sector and repealing Directive 2005...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n14.6.2019 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 158/1\n\n\n\n\n\n\nREGULATION (EU) 2019/941 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof 5 ...,en official journal of the european union regulation of the european parliament and of the council of june on risk preparedness in the electricity sector and repealing directive ec the european pa...,regulation of the european parliament and of the council of june on risk preparedness in the electricity sector and repealing directive ec regulation of the european parliament and of the council ...
3,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nRegulation (EU) 2019/943 of the European Parliament and of the Council of 5 June 2019 on the internal market for electricity (Text with EEA relevance.)\nRegulati...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n14.6.2019 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 158/54\n\n\n\n\n\n\nREGULATION (EU) 2019/943 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof 5...,en official journal of the european union regulation of the european parliament and of the council of june on the internal market for electricity the european parliament and the council of the eur...,regulation of the european parliament and of the council of june on the internal market for electricity regulation of the european parliament and of the council of june on the internal market for ...
4,,\n\n\t\t\t\t\t\t\t​\n\t\t\t\t\t\t\nRegulation (EU) 2019/942 of the European Parliament and of the Council of 5 June 2019 establishing a European Union Agency for the Cooperation of Energy Regulato...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n14.6.2019 \n\n\nEN\n\n\nOfficial Journal of the European Union\n\n\nL 158/22\n\n\n\n\n\n\nREGULATION (EU) 2019/942 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof 5...,en official journal of the european union regulation of the european parliament and of the council of june establishing european union agency for the cooperation of energy regulators the european ...,regulation of the european parliament and of the council of june establishing european union agency for the cooperation of energy regulators regul